In [1]:
# Overall File
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import csv
from scipy.stats import multivariate_normal
# define data paths
file1 = './../data/41467_2024_46346_MOESM4_ESM.csv'
file2 = './../data/41467_2024_46346_MOESM7_ESM.csv'
# read chemical data and merge it with only the overall score
chemData = pd.read_csv(file1)
dataWithOverall = pd.read_csv(file2)
overallAndId = dataWithOverall[['beer','overall']]
# now merge and remove duplicates
merged_df = chemData.merge(overallAndId, on = ['beer'], how='inner')
# Convert string in tasting_category_fine using factorize
merged_df['tasting_category_fine'], category_mapping = pd.factorize(merged_df['tasting_category_fine'])
# Remove NAs
na_counts_per_column = merged_df.isna().sum()
na_counts_per_column = na_counts_per_column[na_counts_per_column > 0]
threshold = 30
# Filter to show only columns with NA counts greater than the threshold
columns_to_drop = na_counts_per_column[na_counts_per_column > threshold].index
# Drop the columns with NA counts greater than the threshold
merged_df_withoutNa = merged_df.drop(columns=columns_to_drop)
merged_df_withoutNa = merged_df_withoutNa.fillna(0)
# drop all 4 SUM columns since we are only interested in the chems themselves
df = merged_df_withoutNa[merged_df_withoutNa.columns.drop(list(merged_df_withoutNa.filter(regex='_sum')))]
df.head(10)


,beer,beer_id,tasting_category_fine,acetaldehyde,CS2,DES,DMS,ethyl_2.methyl_butyrate,ethyl_acetate,ethyl_decanoate,...,Z.Z.geranyl.linalool,ethyl.hexadecanoate,manool.oxide,X13.epi.manool.oxide,isopropyl.hexadecanoate,manool,ethyl.octadecanoate,ethyl.pentanoate,X2.ethyl.3.methylpyrazine,overall
0,10,5410228202929,0,-0.641741,-2.146292,-1.905267,0.046211,-3.195520,-0.599350,-1.975309,...,-1.817819,-1.817819,3.702195,4.360243,-1.817819,5.721204,4.438951,0.014728,0.054353,-0.467852
1,86,5410783031019,0,-0.174824,-2.146292,-0.257098,0.347241,-3.195520,-0.507254,-1.755970,...,3.576605,-1.817819,4.133651,4.512589,4.323292,5.810501,4.247826,0.023167,0.001400,-0.994806
2,2,5410228142003,0,0.511067,-2.146292,-1.905267,0.898604,-3.195520,-0.134514,-1.893639,...,-1.817819,-1.817819,3.966730,4.394267,-1.817819,5.585529,3.598293,0.017571,0.000699,-1.528544
3,83,54055520,0,0.249793,-2.146292,-1.905267,0.782126,-3.195520,-0.233874,-2.086981,...,-1.817819,-1.817819,3.812901,4.174304,4.088807,5.204583,4.498781,0.016959,0.001101,-0.934138
4,40,5410228141181,0,-1.902829,-0.756987,-1.347270,0.553356,-3.195520,0.207751,-1.713095,...,-1.817819,6.484789,1.618481,4.742899,4.651536,6.515499,4.789220,0.117921,0.000492,-1.735965
5,247,5411081000264,1,0.404802,-2.146292,-1.905267,2.108738,-2.850781,0.659248,-2.161466,...,3.552430,-1.817819,3.905719,-1.817819,4.323532,5.908192,5.322008,0.023274,0.049058,-0.800698
6,48,8711406995211,2,0.891771,-2.146292,0.028093,1.399666,-2.512155,1.577115,-0.704521,...,5.252547,6.207206,4.051992,3.271554,4.647266,5.923461,5.228784,0.017974,0.009970,-0.259324
7,165,5411081000363,1,-0.243322,-2.146292,-0.556487,0.819550,-2.367037,0.465608,-2.674690,...,4.268160,-1.817819,4.048183,-1.817819,4.693955,5.858582,5.262941,0.022817,0.071466,-0.250890
8,124,5412186002658,3,0.087730,-2.146292,-0.336360,1.155214,-3.195520,0.949666,-1.944622,...,3.016781,5.598731,2.101812,4.129525,-1.817819,5.889627,-1.817819,0.034472,0.001180,0.230788
9,207,5411223030036,1,0.567943,-2.146292,-0.521224,1.085299,-0.539583,0.786460,-2.195861,...,3.853942,-1.817819,4.059355,-1.817819,4.724163,6.081144,4.399498,0.021659,0.513474,-0.677687


In [33]:
def display_column_names(dataframe):
    """
    Display all column names of the DataFrame.

    Parameters:
    dataframe (pd.DataFrame): The input DataFrame.
    """
    column_names = dataframe.columns
    print("Column names:")
    for col in column_names:
        print(col)



display_column_names(df)

Column names:
beer
beer_id
tasting_category_fine
acetaldehyde
CS2
DES
DMS
ethyl_2.methyl_butyrate
ethyl_acetate
ethyl_decanoate
ethyl_hexanoate
ethyl_isovalerate
ethyl_octanoate
ethyl_propionate
H2S
isoamyl_alcohol
isobutanol
isobutyl_acetate
isopentyl_acetate
MeSH
octyl_acetate
phenethyl_acetate
phenethyl_alcohol
X4VG
SO2.mg.L.
acetic_acid.g.L.
ammonia.mg.L.
color.EBC.
betaglucan.mg.L.
lactic_acid.mg.L.
glycerol
iron
pH
protein.g.L.
bitterness
GFS
ethanol..v.v.
CO2.PSI.
unfermentables
kcalperc
X1.propanol
X2.3.butanedione
X3.methylbutanal
propylene.glycol
propyl.acetate
n.pentanol
X2.methyl.1.butanol
isopentyl.formate
ethyl.isobutyrate
X3.methyl.2.butenal
X2.methyltetrahydrofuran.3.one
ethyl.butyrate
butyl.acetate
methylpyrazine
furfural
ethyl.lactate
X3Z.hexenol
n.hexanol
styrene
X2.furanmethanol
X3.methyl.2.hexanol
X3.furanmethanol
X2.acetylfuran
X2.6.dimethyl.pyrazine
amyl.acetate
ethyl.tiglate
benzaldehyde
X5.methylfurfural
n.heptanol
X1.octen.3.ol
X3.methylthio.propanol
isomaltol

In [32]:
def generate_samples_with_bad(dataframe, n_samples, bad_columns, percentile=10):
    sampled_data = {}
    
    # Determine numeric and categorical columns
    num_cols = dataframe.select_dtypes(include=['number']).columns
    cat_cols = dataframe.select_dtypes(include=['category', 'object']).columns
    
    # Sample integer values for categorical columns
    for col in dataframe.columns:
        if col in cat_cols:
            unique_values = dataframe[col].dropna().unique()
            if unique_values.any():
                sampled_data[col] = np.random.choice(unique_values, size=n_samples)
            else:
                sampled_data[col] = np.nan
        elif col in num_cols:
            col_data = dataframe[col].dropna()
            mu, sigma = stats.norm.fit(col_data)
            if col in bad_columns:
                threshold = np.percentile(col_data, percentile)
                left_tail = col_data[col_data <= threshold]
                sampled_data[col] = np.random.choice(left_tail, size=n_samples, replace=True)
            else:
                samples = stats.norm(loc=mu, scale=sigma).rvs(size=n_samples)
                # Filter out -inf and inf values
                samples = samples[np.isfinite(samples)]
                # Ensure enough samples are drawn
                while len(samples) < n_samples:
                    additional_samples = stats.norm(loc=mu, scale=sigma).rvs(size=(n_samples - len(samples)))
                    additional_samples = additional_samples[np.isfinite(additional_samples)]
                    samples = np.concatenate((samples, additional_samples))
                sampled_data[col] = samples[:n_samples]
    
    return pd.DataFrame(sampled_data)

In [37]:
min_values = df.min()
max_values = df.max()
avg_values = df.mean()

print(min_values['ethanol..v.v.'])
print(max_values['ethanol..v.v.'])
print(avg_values['ethanol..v.v.'])

-1.455931956
1.06595298
0.8039263130039999


In [34]:
# Remove Beer/Beer_Id/tasting_category and overall from newly generated sample
# Why tasting category? If we don't remove it we get in trouble because we would have a dependency between ethanol and alcohol free beers
# since the category itself is determined by chemical properties we are fine with omiting it from the result
rand_df = df.drop(columns=['beer','beer_id','overall'])
# Use name of columns that perform best in our GP analysis, start with what paper people found
rand_sample = generate_samples_with_bad(rand_df, 500, bad_columns=['ethyl_acetate','ethanol..v.v.', 'ethyl_octanoate','ethyl.phenylacetate','protein.g.L.', 'lactic_acid.mg.L.'],percentile=20)
rand_sample.head(10)

,tasting_category_fine,acetaldehyde,CS2,DES,DMS,ethyl_2.methyl_butyrate,ethyl_acetate,ethyl_decanoate,ethyl_hexanoate,ethyl_isovalerate,...,methyl.hexadecanoate,Z.Z.geranyl.linalool,ethyl.hexadecanoate,manool.oxide,X13.epi.manool.oxide,isopropyl.hexadecanoate,manool,ethyl.octadecanoate,ethyl.pentanoate,X2.ethyl.3.methylpyrazine
0,11.484709,0.833820,-1.085630,-1.295951,1.422158,-1.979490,1.192901,-1.222155,-1.068566,-4.233990,...,4.221430,2.190147,11.238513,4.147901,4.431221,0.412367,0.148590,5.064044,0.128116,-0.003023
1,7.861972,0.467301,-2.380854,-0.372815,1.945702,-2.430756,1.210230,-0.973792,-0.318121,-0.896117,...,5.880250,5.885295,6.995387,2.975560,0.885097,2.630698,4.944574,4.735460,-0.059529,-0.172884
2,14.294664,0.032701,-2.066107,-0.127100,1.286897,-2.710230,1.051042,-0.787627,-0.293660,-2.368600,...,6.130992,3.295943,1.322984,4.367856,1.715621,3.218469,6.611361,4.677169,0.507631,-0.037314
3,5.906708,0.270167,-1.344637,-0.593707,1.872393,-2.368777,1.025313,-0.643033,-0.883447,-3.511999,...,2.098283,6.155451,11.841041,2.853938,3.592049,0.644029,3.642527,6.477312,0.068439,-0.088285
4,18.075591,0.601380,-2.081504,-1.905955,2.020462,-3.351130,1.142983,-0.364520,-0.891570,-4.219160,...,5.808799,3.340343,5.760343,2.881258,6.872955,7.452291,4.262898,8.220855,0.030728,0.229248
5,1.807839,0.773260,-1.843644,-1.529781,1.522332,-1.998928,0.786460,-1.052956,-0.292895,-0.889866,...,7.176344,1.867798,2.807321,3.093313,3.918783,-0.753338,6.064323,1.671816,0.106197,0.067746
6,5.886703,0.506836,-1.533447,-0.730872,1.390228,-3.396698,1.052084,-1.371770,-0.660222,-2.797873,...,2.579266,4.807969,4.778146,5.308094,1.152780,-0.492476,4.709883,4.368184,-0.339640,0.092700
7,3.629632,0.298018,0.492961,-1.128423,0.903072,-3.153427,1.168692,-1.446677,-0.364996,-2.098542,...,4.612307,3.480544,7.211566,3.108278,3.693865,-2.192759,3.756100,4.618613,0.186862,-0.028542
8,7.314748,0.460001,-1.972903,0.873580,0.918495,-2.810805,0.659248,-0.395747,-0.674054,-2.574739,...,4.563202,7.091516,5.596980,4.220528,0.931366,-2.852632,6.445659,2.607150,-0.107217,0.051758
9,10.214056,0.929587,-1.350041,0.220306,1.255857,-1.802847,1.177058,-1.187813,-1.281708,-2.560235,...,3.576895,5.301798,3.779122,5.321285,4.823845,6.223187,4.173611,5.553916,0.114893,0.101335


In [36]:
min_values_rand = rand_sample.min()
max_values_rand = rand_sample.max()
avg_values_rand = rand_sample.mean()

print(min_values_rand['ethanol..v.v.'])
print(max_values_rand['ethanol..v.v.'])
print(avg_values_rand['ethanol..v.v.'])

-1.455931956
0.721398376
0.5117974392019999


AttributeError: 'DataFrame' object has no attribute 'avg'